# Import Dependancies

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

connection_string = f"group5:Group5Project@nbastats2.c4lr0vlcqzd5.us-east-1.rds.amazonaws.com:5432/NBA_Stats"
engine = create_engine(f'postgresql://{connection_string}')


In [2]:
# Read the csv file into a pandas DataFrame

all_rookies_df = pd.read_excel('Data/ROY Data/all_rookies_1990_2020.xlsx')
all_rookies_df.drop(columns=['Unnamed: 0','Rk','Debut','Age','Yrs'],inplace=True)
all_rookies_df["Player"] = all_rookies_df.Player.str.replace('*', '')

# Create a dataframe of rookies for the 2019-20
rookies_2019_20_df = all_rookies_df.loc[all_rookies_df["season"] == "2019-20"]

# Remove records for the 2019-20 season from all rookies dataframe
all_rookies_df = all_rookies_df.loc[all_rookies_df["season"] != "2019-20"]
#all_rookies_df = all_rookies_df.loc[all_rookies_df["season"] != "2018-19"]
#all_rookies_df = all_rookies_df.loc[all_rookies_df["season"] != "2017-18"]
#all_rookies_df = all_rookies_df.loc[all_rookies_df["season"] != "2016-17"]


In [3]:
# Read Rookie of the Year candidates from csv
roy_votes_df = pd.read_excel('Data/ROY Data/all_roy_candidates_1990_2020.xlsx')

# Restructure the dataframe to contain only player and season
roy_votes_df = roy_votes_df[["Player", "season","Share"]]

# Add a column and set to 1 as Rookie of the Year candidate
roy_votes_df["ROYCandidate"] = 1

#roy_votes_df = roy_votes_df.loc[roy_votes_df["season"] != "2018-19"]
#roy_votes_df = roy_votes_df.loc[roy_votes_df["season"] != "2017-18"]
#roy_votes_df = roy_votes_df.loc[roy_votes_df["season"] != "2016-17"]

#roy_votes_df

In [4]:
# left merge all rookies and candidates
rookies_2019_20_df = pd.merge(rookies_2019_20_df, roy_votes_df, how='left', on=["Player","season"])
# Set NaN to 0
rookies_2019_20_df["3P%"] = rookies_2019_20_df["3P%"].fillna(0)
rookies_2019_20_df["FG%"] = rookies_2019_20_df["FG%"].fillna(0)

# Add additional per game average columns
rookies_2019_20_df["BLKPG"] = rookies_2019_20_df["BLK"]/rookies_2019_20_df["G"]
rookies_2019_20_df["STLPG"] = rookies_2019_20_df["STL"]/rookies_2019_20_df["G"]
rookies_2019_20_df["TOVPG"] = rookies_2019_20_df["TOV"]/rookies_2019_20_df["G"]


In [5]:
# left merge all rookies and candidates
all_rookies_df = pd.merge(all_rookies_df, roy_votes_df, how='left', on=["Player","season"])

# Set all ROYCandidate to 0 if it is Nan - Not a candidate
all_rookies_df["ROYCandidate"] = all_rookies_df["ROYCandidate"].fillna(0)
all_rookies_df["3P%"] = all_rookies_df["3P%"].fillna(0)
all_rookies_df["FG%"] = all_rookies_df["FG%"].fillna(0)

all_rookies_df["BLKPG"] = all_rookies_df["BLK"]/all_rookies_df["G"]
all_rookies_df["STLPG"] = all_rookies_df["STL"]/all_rookies_df["G"]
all_rookies_df["TOVPG"] = all_rookies_df["TOV"]/all_rookies_df["G"]



In [6]:
# Read csv file with advnaced statistics
advanced_df = pd.read_csv("AdvancedStats.csv")

# drop column player as it is not needed
advanced_df.drop(columns="Player", inplace=True)
# rename PlayerName to Player to make merge easier
advanced_df["Player"] = advanced_df["PlayerName"]

# Drop duplicate records for players played on multiple teams during a season, keep the Totals
advanced_df = advanced_df.drop_duplicates(subset=['Player','season'], keep='first')
#advanced_df = advanced_df.loc[advanced_df["Tm"] != "TOT"]

# Merge All Rookies and Advanced Statistics
all_rookies_df = pd.merge(all_rookies_df, advanced_df, how='inner', on=["Player","season"])

# Set values to 0 for NaN
all_rookies_df["PER"] = all_rookies_df["PER"].fillna(0)
all_rookies_df["WS"] = all_rookies_df["WS"].fillna(0)
all_rookies_df["VORP"] = all_rookies_df["VORP"].fillna(0)

# Merge 2019-20 Rookies and Adbanced Statistics
rookies_2019_20_df = pd.merge(rookies_2019_20_df, advanced_df, how='inner', on=["Player","season"])

# Set values to 0 for NaN
rookies_2019_20_df["PER"] = rookies_2019_20_df["PER"].fillna(0)
rookies_2019_20_df["WS"] = rookies_2019_20_df["WS"].fillna(0)
rookies_2019_20_df["VORP"] = rookies_2019_20_df["VORP"].fillna(0)

# Data Pre-processing

Includes:

* Converting categorical data
* Feature selection
* Scaling and transforming data

In [7]:
# Assign X (features) and y (target)
# X and y to be used for trin/test
# Xr is current rookies to predict if Rookie of the year candidate

X = all_rookies_df[['MP.1','PER','WS','VORP']]
Xr = rookies_2019_20_df[['MP.1','PER','WS','VORP']]

#X = all_rookies_df[['MP.1', 'PTS.1', 'TRB.1', 'AST.1','BLKPG','STLPG']]
#Xr = rookies_2019_20_df[['MP.1', 'PTS.1', 'TRB.1', 'AST.1','BLKPG','STLPG']]

#X = all_rookies_df[['MP_x','FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB','TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS']]
#Xr = rookies_2019_20_df[['MP_x','FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB','TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS']]


y = all_rookies_df["ROYCandidate"]

# Reshape target if necessary (some functions in sklearn require 2-d arrays)
y = y.values.reshape(-1, 1)

In [8]:
# Split the data into training and testing
# Default split is 80% testing, 20% training
# Setting random_state is not necessary but makes results reproducible

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
# Create a StandardScater model and fit it to the training data
# StandardScaler makes data normally distributed

X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

In [10]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
Xr_test_scaled = X_scaler.transform(Xr)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [11]:
# Load the model
from tensorflow.keras.models import load_model
deep_model = load_model("rookie_trained.h5")
#deep_model = load_model("rookie_trained_trad.h5")

name_list=[]
season_list=[]
team_list=[]
model_list=[]
ynew = deep_model.predict_classes(Xr_test_scaled)
for i in range(len(Xr_test_scaled)):
    if (ynew[i] == 1):
        print(i, rookies_2019_20_df.iloc[i,0],rookies_2019_20_df.iloc[i,33],rookies_2019_20_df.iloc[i,24])
        name_list.append(rookies_2019_20_df.iloc[i,0])
        season_list.append(rookies_2019_20_df.iloc[i,24])
        team_list.append(rookies_2019_20_df.iloc[i,31])
        model_list.append("Basic Stats Deep Learning")
        rookies_2019_20_df.iloc[i,26] = 1

16 Brandon Clarke MEM 2019-20
35 Rui Hachimura WAS 2019-20
39 Jaxson Hayes NOP 2019-20
72 Ja Morant MEM 2019-20
78 Kendrick Nunn MIA 2019-20
105 P.J. Washington CHO 2019-20
112 Zion Williamson NOP 2019-20


In [12]:
candidates_df = all_rookies_df.loc[all_rookies_df["ROYCandidate"]==1]


In [13]:
candidates_2019_20_df = rookies_2019_20_df.loc[rookies_2019_20_df["ROYCandidate"]==1]


In [14]:
X = candidates_df[['MP.1','PER','WS','VORP']]
Xr = candidates_2019_20_df[['MP.1','PER','WS','VORP']]

#X = all_rookies_df[['MP.1', 'PTS.1', 'TRB.1', 'AST.1','BLKPG','STLPG']]
#Xr = rookies_2019_20_df[['MP.1', 'PTS.1', 'TRB.1', 'AST.1','BLKPG','STLPG']]

#X = all_rookies_df[['MP_x','FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB','TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS']]
#Xr = rookies_2019_20_df[['MP_x','FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB','TRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS']]


y = candidates_df["Share"]

# Reshape target if necessary (some functions in sklearn require 2-d arrays)
y = y.values.reshape(-1, 1)

In [15]:
# Split the data into training and testing
# Default split is 80% testing, 20% training
# Setting random_state is not necessary but makes results reproducible

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [16]:
# Create a StandardScater model and fit it to the training data
# StandardScaler makes data normally distributed
from sklearn.preprocessing import MinMaxScaler
#X_scaler = StandardScaler().fit(X_train)
#y_scaler = StandardScaler().fit(y_train)
X_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(y_train)

In [17]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
Xr_test_scaled = X_scaler.transform(Xr)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

deep_model = Sequential()
deep_model.add(Dense(units=8, activation='relu', input_dim=4))
deep_model.add(Dense(units=6, activation='relu'))
deep_model.add(Dense(units=1, activation='linear'))

deep_model.summary()

deep_model.compile(optimizer='adam',
                   loss='mse',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=1000,
    shuffle=True,
    verbose=2   
)


model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled,y_test_scaled, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 40        
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 54        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 7         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 142 samples
Epoch 1/1000
142/142 - 1s - loss: 0.1532 - accuracy: 0.1268
Epoch 2/1000
142/142 - 0s - loss: 0.1465 - accuracy: 0.1268
Epoch 3/1000
142/142 - 0s - loss: 0.1411 - accuracy: 0.1268
Epoch 4/1000
142/142 - 0s - loss: 0.1369 - accuracy: 0.1268
Epoch 5/1000
142/142 - 0s - loss: 0.1340 - accuracy: 0.1268
Epoch 6/1000
142/142 - 0s - loss: 0.1315 - accuracy: 0.1268
Epoch 7

Epoch 123/1000
142/142 - 0s - loss: 0.0733 - accuracy: 0.1268
Epoch 124/1000
142/142 - 0s - loss: 0.0730 - accuracy: 0.1268
Epoch 125/1000
142/142 - 0s - loss: 0.0729 - accuracy: 0.1268
Epoch 126/1000
142/142 - 0s - loss: 0.0729 - accuracy: 0.1268
Epoch 127/1000
142/142 - 0s - loss: 0.0726 - accuracy: 0.1268
Epoch 128/1000
142/142 - 0s - loss: 0.0728 - accuracy: 0.1268
Epoch 129/1000
142/142 - 0s - loss: 0.0723 - accuracy: 0.1268
Epoch 130/1000
142/142 - 0s - loss: 0.0721 - accuracy: 0.1268
Epoch 131/1000
142/142 - 0s - loss: 0.0720 - accuracy: 0.1268
Epoch 132/1000
142/142 - 0s - loss: 0.0719 - accuracy: 0.1268
Epoch 133/1000
142/142 - 0s - loss: 0.0715 - accuracy: 0.1268
Epoch 134/1000
142/142 - 0s - loss: 0.0718 - accuracy: 0.1268
Epoch 135/1000
142/142 - 0s - loss: 0.0714 - accuracy: 0.1268
Epoch 136/1000
142/142 - 0s - loss: 0.0712 - accuracy: 0.1268
Epoch 137/1000
142/142 - 0s - loss: 0.0710 - accuracy: 0.1268
Epoch 138/1000
142/142 - 0s - loss: 0.0711 - accuracy: 0.1268
Epoch 13

Epoch 256/1000
142/142 - 0s - loss: 0.0603 - accuracy: 0.1338
Epoch 257/1000
142/142 - 0s - loss: 0.0600 - accuracy: 0.1338
Epoch 258/1000
142/142 - 0s - loss: 0.0597 - accuracy: 0.1338
Epoch 259/1000
142/142 - 0s - loss: 0.0600 - accuracy: 0.1338
Epoch 260/1000
142/142 - 0s - loss: 0.0596 - accuracy: 0.1338
Epoch 261/1000
142/142 - 0s - loss: 0.0598 - accuracy: 0.1338
Epoch 262/1000
142/142 - 0s - loss: 0.0595 - accuracy: 0.1338
Epoch 263/1000
142/142 - 0s - loss: 0.0598 - accuracy: 0.1338
Epoch 264/1000
142/142 - 0s - loss: 0.0601 - accuracy: 0.1338
Epoch 265/1000
142/142 - 0s - loss: 0.0595 - accuracy: 0.1338
Epoch 266/1000
142/142 - 0s - loss: 0.0597 - accuracy: 0.1338
Epoch 267/1000
142/142 - 0s - loss: 0.0602 - accuracy: 0.1338
Epoch 268/1000
142/142 - 0s - loss: 0.0597 - accuracy: 0.1338
Epoch 269/1000
142/142 - 0s - loss: 0.0593 - accuracy: 0.1338
Epoch 270/1000
142/142 - 0s - loss: 0.0594 - accuracy: 0.1338
Epoch 271/1000
142/142 - 0s - loss: 0.0591 - accuracy: 0.1338
Epoch 27

142/142 - 0s - loss: 0.0558 - accuracy: 0.1338
Epoch 389/1000
142/142 - 0s - loss: 0.0561 - accuracy: 0.1338
Epoch 390/1000
142/142 - 0s - loss: 0.0557 - accuracy: 0.1338
Epoch 391/1000
142/142 - 0s - loss: 0.0557 - accuracy: 0.1338
Epoch 392/1000
142/142 - 0s - loss: 0.0557 - accuracy: 0.1338
Epoch 393/1000
142/142 - 0s - loss: 0.0558 - accuracy: 0.1338
Epoch 394/1000
142/142 - 0s - loss: 0.0558 - accuracy: 0.1338
Epoch 395/1000
142/142 - 0s - loss: 0.0559 - accuracy: 0.1338
Epoch 396/1000
142/142 - 0s - loss: 0.0556 - accuracy: 0.1338
Epoch 397/1000
142/142 - 0s - loss: 0.0559 - accuracy: 0.1338
Epoch 398/1000
142/142 - 0s - loss: 0.0556 - accuracy: 0.1338
Epoch 399/1000
142/142 - 0s - loss: 0.0556 - accuracy: 0.1338
Epoch 400/1000
142/142 - 0s - loss: 0.0556 - accuracy: 0.1408
Epoch 401/1000
142/142 - 0s - loss: 0.0556 - accuracy: 0.1338
Epoch 402/1000
142/142 - 0s - loss: 0.0555 - accuracy: 0.1408
Epoch 403/1000
142/142 - 0s - loss: 0.0558 - accuracy: 0.1408
Epoch 404/1000
142/142 

Epoch 521/1000
142/142 - 0s - loss: 0.0538 - accuracy: 0.1408
Epoch 522/1000
142/142 - 0s - loss: 0.0537 - accuracy: 0.1408
Epoch 523/1000
142/142 - 0s - loss: 0.0541 - accuracy: 0.1408
Epoch 524/1000
142/142 - 0s - loss: 0.0537 - accuracy: 0.1408
Epoch 525/1000
142/142 - 0s - loss: 0.0537 - accuracy: 0.1408
Epoch 526/1000
142/142 - 0s - loss: 0.0536 - accuracy: 0.1408
Epoch 527/1000
142/142 - 0s - loss: 0.0537 - accuracy: 0.1408
Epoch 528/1000
142/142 - 0s - loss: 0.0537 - accuracy: 0.1408
Epoch 529/1000
142/142 - 0s - loss: 0.0538 - accuracy: 0.1408
Epoch 530/1000
142/142 - 0s - loss: 0.0537 - accuracy: 0.1408
Epoch 531/1000
142/142 - 0s - loss: 0.0536 - accuracy: 0.1408
Epoch 532/1000
142/142 - 0s - loss: 0.0536 - accuracy: 0.1408
Epoch 533/1000
142/142 - 0s - loss: 0.0537 - accuracy: 0.1408
Epoch 534/1000
142/142 - 0s - loss: 0.0535 - accuracy: 0.1408
Epoch 535/1000
142/142 - 0s - loss: 0.0540 - accuracy: 0.1408
Epoch 536/1000
142/142 - 0s - loss: 0.0535 - accuracy: 0.1408
Epoch 53

Epoch 654/1000
142/142 - 0s - loss: 0.0525 - accuracy: 0.1408
Epoch 655/1000
142/142 - 0s - loss: 0.0530 - accuracy: 0.1408
Epoch 656/1000
142/142 - 0s - loss: 0.0528 - accuracy: 0.1408
Epoch 657/1000
142/142 - 0s - loss: 0.0526 - accuracy: 0.1408
Epoch 658/1000
142/142 - 0s - loss: 0.0527 - accuracy: 0.1408
Epoch 659/1000
142/142 - 0s - loss: 0.0530 - accuracy: 0.1408
Epoch 660/1000
142/142 - 0s - loss: 0.0524 - accuracy: 0.1408
Epoch 661/1000
142/142 - 0s - loss: 0.0529 - accuracy: 0.1408
Epoch 662/1000
142/142 - 0s - loss: 0.0529 - accuracy: 0.1408
Epoch 663/1000
142/142 - 0s - loss: 0.0526 - accuracy: 0.1408
Epoch 664/1000
142/142 - 0s - loss: 0.0531 - accuracy: 0.1408
Epoch 665/1000
142/142 - 0s - loss: 0.0531 - accuracy: 0.1408
Epoch 666/1000
142/142 - 0s - loss: 0.0525 - accuracy: 0.1408
Epoch 667/1000
142/142 - 0s - loss: 0.0525 - accuracy: 0.1408
Epoch 668/1000
142/142 - 0s - loss: 0.0524 - accuracy: 0.1408
Epoch 669/1000
142/142 - 0s - loss: 0.0525 - accuracy: 0.1408
Epoch 67

Epoch 787/1000
142/142 - 0s - loss: 0.0518 - accuracy: 0.1408
Epoch 788/1000
142/142 - 0s - loss: 0.0522 - accuracy: 0.1408
Epoch 789/1000
142/142 - 0s - loss: 0.0516 - accuracy: 0.1408
Epoch 790/1000
142/142 - 0s - loss: 0.0516 - accuracy: 0.1408
Epoch 791/1000
142/142 - 0s - loss: 0.0517 - accuracy: 0.1408
Epoch 792/1000
142/142 - 0s - loss: 0.0516 - accuracy: 0.1408
Epoch 793/1000
142/142 - 0s - loss: 0.0517 - accuracy: 0.1408
Epoch 794/1000
142/142 - 0s - loss: 0.0518 - accuracy: 0.1408
Epoch 795/1000
142/142 - 0s - loss: 0.0518 - accuracy: 0.1408
Epoch 796/1000
142/142 - 0s - loss: 0.0519 - accuracy: 0.1408
Epoch 797/1000
142/142 - 0s - loss: 0.0516 - accuracy: 0.1408
Epoch 798/1000
142/142 - 0s - loss: 0.0516 - accuracy: 0.1408
Epoch 799/1000
142/142 - 0s - loss: 0.0520 - accuracy: 0.1408
Epoch 800/1000
142/142 - 0s - loss: 0.0516 - accuracy: 0.1408
Epoch 801/1000
142/142 - 0s - loss: 0.0517 - accuracy: 0.1408
Epoch 802/1000
142/142 - 0s - loss: 0.0520 - accuracy: 0.1408
Epoch 80

Epoch 920/1000
142/142 - 0s - loss: 0.0507 - accuracy: 0.1408
Epoch 921/1000
142/142 - 0s - loss: 0.0508 - accuracy: 0.1408
Epoch 922/1000
142/142 - 0s - loss: 0.0507 - accuracy: 0.1408
Epoch 923/1000
142/142 - 0s - loss: 0.0508 - accuracy: 0.1408
Epoch 924/1000
142/142 - 0s - loss: 0.0507 - accuracy: 0.1408
Epoch 925/1000
142/142 - 0s - loss: 0.0508 - accuracy: 0.1408
Epoch 926/1000
142/142 - 0s - loss: 0.0511 - accuracy: 0.1408
Epoch 927/1000
142/142 - 0s - loss: 0.0509 - accuracy: 0.1408
Epoch 928/1000
142/142 - 0s - loss: 0.0507 - accuracy: 0.1408
Epoch 929/1000
142/142 - 0s - loss: 0.0508 - accuracy: 0.1408
Epoch 930/1000
142/142 - 0s - loss: 0.0508 - accuracy: 0.1408
Epoch 931/1000
142/142 - 0s - loss: 0.0509 - accuracy: 0.1408
Epoch 932/1000
142/142 - 0s - loss: 0.0508 - accuracy: 0.1408
Epoch 933/1000
142/142 - 0s - loss: 0.0508 - accuracy: 0.1408
Epoch 934/1000
142/142 - 0s - loss: 0.0509 - accuracy: 0.1408
Epoch 935/1000
142/142 - 0s - loss: 0.0509 - accuracy: 0.1408
Epoch 93

In [19]:
# Load the model
from tensorflow.keras.models import load_model
deep_model = load_model("rookie_score.h5")

ynew = deep_model.predict(Xr_test_scaled)
player_list = []
score_list = []
team_list = []
season_list = []
for i in range(len(Xr_test_scaled)):
    print(ynew[i][0],candidates_2019_20_df.iloc[i,0])
    player_list.append(candidates_2019_20_df.iloc[i,0])
    score_list.append(ynew[i][0])
    team_list.append(candidates_2019_20_df.iloc[i,33])
    season_list.append(candidates_2019_20_df.iloc[i,24])

0.07175136 Brandon Clarke
0.103075325 Rui Hachimura
0.023658164 Jaxson Hayes
0.31290284 Ja Morant
0.10739012 Kendrick Nunn
0.07011481 P.J. Washington
0.28849524 Zion Williamson


In [20]:
df = pd.DataFrame({'Player': player_list,'season':season_list,'Tm':team_list, 'score': score_list})
df.sort_values("score",ascending=False,inplace=True)
df.reset_index(drop=True)

,Player,season,Tm,score
0,Ja Morant,2019-20,MEM,0.312903
1,Zion Williamson,2019-20,NOP,0.288495
2,Kendrick Nunn,2019-20,MIA,0.107390
3,Rui Hachimura,2019-20,WAS,0.103075
4,Brandon Clarke,2019-20,MEM,0.071751
5,P.J. Washington,2019-20,CHO,0.070115
6,Jaxson Hayes,2019-20,NOP,0.023658


In [21]:
# Save the model
#deep_model.save("rookie_score.h5")

In [22]:
# save to DB
df.to_sql(name='roy_predictions_rank', con=engine, if_exists='append', index=False)